In [ ]:
# ----------------------------------------------------------------------------------------------------------------------
# Importing libraries and necessary functions
# ----------------------------------------------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import pandas_dedupe
import os
from tqdm import tqdm


In [ ]:
""""
------------------------------------------------------------------------------------------------------------------------
DeDuping Tool
------------------------------------------------------------------------------------------------------------------------
Date: January 2022
Considerations:
    -
Authors:
        Clara Graham                        clara.graham@accenture.com
        Lucas Asein                         lucas.asein@accenture.com
        Shameez Manji                       shameez.manji@accenture.com
        Ignacio Brottier González           ignacio.brottier@accenture.com
        Arielle Jaraba Serrano              arielle.serrano@accenture.com
"""

# ----------------------------------------------------------------------------------------------------------------------
# Importing libraries and necessary functions
# ----------------------------------------------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import pandas_dedupe
import os
from tqdm import tqdm


#  ----------------------------------------------------------------------------------------------------------------------
# Model functions
# # ----------------------------------------------------------------------------------------------------------------------


def dupe_train(df_sample):
    # 1. Delete old training file
    ans = yes_no_answer('Would you like to retrain the model?')
    if ans:
        # Try to delete the file ##
        try:
            os.remove('dedupe_dataframe_learned_settings')
            print('\tSettings file removed')
        except OSError as e:  ## if failed, report it back to the user ##
            print("Error: %s - %s." % (e.filename, e.strerror))
        try:
            os.remove('dedupe_dataframe_training.json')
            print('\tJson file removed')
        except OSError as e:  ## if failed, report it back to the user ##
            print("Error: %s - %s." % (e.filename, e.strerror))

    print(f'\n\tSample Shape: {df_sample.shape}')
    print('\tSample Head\n')
    print(df_sample.head())

    # 2. Select columns for training
    columns_list = [select_col('Enter the column indexes to use when training the model', df_sample, False)]
    print(f'Selected Columns: {columns_list}')
    not_done = yes_no_answer('Do you want to add more columns?')
    while not_done:
        columns_list.append(select_col('Enter additional columns to train the model', df_sample, False))
        columns_list = list(set(columns_list))
        print(f'Selected Columns: {columns_list}')
        if len(columns_list) < len(list(df_sample.columns)):
            not_done = yes_no_answer('Do you want to add more columns?')
        else:
            not_done = True
    print(f'\nColumns for clustering will be {columns_list}\n')

    # 3. Clean Data
    for c in range(0, len(columns_list)):
        df_sample[columns_list[c]] = df_sample[columns_list[c]].astype('str')
        df_sample[columns_list[c]] = df_sample[columns_list[c]].str.strip()
        df_sample[columns_list[c]] = df_sample[columns_list[c]].str.replace('\n', ' ')

    # 4. Run Model
    print('Training clustering model')
    df_train = pandas_dedupe.dedupe_dataframe(df_sample, columns_list)

    # Sort values by cluster_id
    df_train.sort_values('cluster id', inplace=True)
    df_train.reset_index(inplace=True)
    df_train.drop(columns=['index'], inplace=True)

    return df_train


#  ----------------------------------------------------------------------------------------------------------------------
# UI/UX functions
# # ----------------------------------------------------------------------------------------------------------------------

def load_file(msg):
    found = False
    while not found:
        file_path = input(msg)
        try:
            found = True
            df = pd.read_csv(file_path)
        except FileNotFoundError:
            print('File not found. Please try again\n')
            found = False
    cols = list(df.columns)

    print(f'\n\tThe file {file_path} was loaded correctly\n')
    print(df.head())
    print('\n')
    return df, cols


def select_col(msg, df, print_df=True):
    print(f'{msg}\n')
    if print_df:
        print(df.tail())
        print()

    cols = list(df.columns)
    options = {i: cols[i] for i in range(len(cols))}
    print(options)
    value = input(f'\n')
    while int(value) not in options.keys():
        print(f'Invalid option - {msg}\n')
        print(options)
        value = input(f'\n')

    ans_ = cols[int(value)]
    print(f'\t Columns Selected: {ans_}')
    if not yes_no_answer('Continue?'):
        select_col(msg, df, False)

    return ans_


def yes_no_answer(msg):
    msg = msg + ' (y) Yes - (n) No\n'
    value = input(msg)
    value = str(value).lower()
    while value not in ['y', 'n']:
        print(value)
        print('Invalid Answer. Please try again')
        value = input(msg)
        value = value.lower()

    ans_ = True if value == 'y' else False
    return ans_


def preview_results(df, n=5):
    ans_ = yes_no_answer('Do you want to preview some results?')

    # Sort values by confidence
    df.sort_values('confidence', inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)

    x = 0
    cluster_id_list = list(df['cluster id'].unique())
    while ans_:
        for i in range(0, n):
            cluster = cluster_id_list[x]
            df_example = df[df['cluster id'] == cluster]

            print(f'\tCluster id: {cluster}')
            print(f'\tCluster confidence: {df_example["confidence"].mean()}\n')

            df_example = df_example.sample(n=min(len(df_example), 5))
            print(df_example)

            x += 1

        ans_ = yes_no_answer('Do you want to preview more results?')


def break_line():
    aux = 100 * "="
    print(f'\n\t\t{aux}\n')


#  ----------------------------------------------------------------------------------------------------------------------
# Main
# # ----------------------------------------------------------------------------------------------------------------------

if __name__ == '__main__':
    print('\t\t\t\tWelcome to DeDupping Tool\n\n')

    # 1. Load Data
    #       Excel files closed warning
    print('All related spreadsheets should be closed')
    #       Load New Data File
    df_data, list_cols = load_file('Please enter the name of the spreadsheet you would like to dedupe. Make sure to add the “.csv” extension.\n')

    break_line()

    # 2. Merge with previous deduped file
    has_base_data = yes_no_answer('Would you like to append the output of this data to an existing deduped file?')
    if has_base_data:
        # Load Normalized Names File
        df_norm_names, list_norm_cols = load_file('Please enter the name of the existing deduped csv file. Make sure to include the “.csv” extension.\n')
        # Select Columns
        #       Select Name Column New Data
        column_name = select_col('Select which column from the new data has the client name', df_data)
        #       Select Unique ID Base Data
        column_norm_id = select_col('Select which column from the base data has the client unique id', df_norm_names)
        #       Select Name Column Base Data
        column_norm_name = select_col('Select which column from the base data has the client name', df_norm_names, False)

        # Maintain original clusters
        #   Append one entry of each old cluster id - name to maintain nomenclature during new clusterization
        print('\nMaintaining original Cluster Id Nomenclature')
        print(f'{len(df_norm_names[column_norm_id].unique())} Unique Cluster Ids found...')
        new_row = {col: "" for col in list_cols}
        for id_aux in tqdm(df_norm_names[column_norm_id].unique()):
            cluster_name = df_norm_names[df_norm_names[column_norm_id] == id_aux].sample(n=1)
            cluster_name = cluster_name.iat[0, list_norm_cols.index(column_norm_name)]
            new_row[column_name] = cluster_name
            df_data = df_data.append(new_row, ignore_index=True)
        print('\tMerging successful')

    break_line()

    ans = False
    while not ans:
        # 3. Train Dedupping model
        df_final = dupe_train(df_data)
        print('\tDedupping model trained successfully\n\n')

        # 4. Preview Results
        preview_results(df_final)
        ans = yes_no_answer('Are the examples OK, or should we retrain?')
        if not ans:
            del df_final

    # 5. Merge with original data
    if has_base_data:
        print('Merging with previous deduped data...')
        df_norm_names[column_norm_name] = df_norm_names[column_norm_name].str.upper()
        df_final[column_name] = df_final[column_name].str.upper()
        aux_old_id = max(df_norm_names[column_norm_id])

        df_join = pd.merge(
            df_norm_names,
            df_final,
            how='outer',
            left_on=[column_norm_name],
            right_on=[column_name]
        )

        id_mapping = {}
        for id_aux in tqdm(df_final['cluster id'].unique()):
            max_old = max(df_join.loc[df_join['cluster id'] == id_aux][column_norm_id])
            if max_old > 0:
                id_mapping[id_aux] = max_old
            else:
                id_mapping[id_aux] = aux_old_id
                aux_old_id += 1
        df_aux = df_final.copy()
        df_aux['cluster id'] = df_aux['cluster id'].map(id_mapping)
        df_aux = df_aux.rename(columns={column_name: column_norm_name, 'cluster id': column_norm_id})
        df_aux = df_aux[[column_norm_name, column_norm_id]]

        df_save = pd.concat([df_aux,df_norm_names]).drop_duplicates().reset_index(drop=True)
        #df_save = pd.concat([df_aux, df_norm_names]).reset_index(drop=True)

        for id_aux in df_norm_names[column_norm_id].unique():
            assert len(df_norm_names[df_norm_names[column_norm_id] == id_aux]) <= len(df_save.loc[df_save[column_norm_id] == id_aux])

    else:
        df_final = df_final[column_name, 'cluster id']
        df_save = df_final.rename(columns={column_name: 'Original Supplier Name', 'cluster id': 'Normalized Name Unique ID'})

    # 6. Save Results
    df_save.to_csv('normalized_names_new.csv', index=False)

    print("\n\nThat would be all, thank you for using the dedupping tool.")
    break_line()
    print('\n\n'
          '\tJanuary 2022\n'
          '\tIgnacio Brottier González           ignacio.brottier@accenture.com\n'
          '\tClara Graham                        clara.graham@accenture.com\n'
          '\tLucas Asein                         lucas.asein@accenture.com\n'
          '\tShameez Manji                       shameez.manji@accenture.com\n'
          '\tArielle Jaraba Serrano              arielle.serrano@accenture.com')
